**Station Information and Update**

We handle each station individually because that is how the algorithms are described and it lets us simulate the parellelism of the true process by telling each station what it's local clock reads at instances A,B,n,B' at the start of each iteration. This is done later in "set_local_times".

In addition to the local times, each station also has a global estimate of time instance n, an id number, it's bias and drift estimates(x), it's estimate covariance (P), the estimates after the cross link updates (psi), and it's nbhr station id's as well as how far away the nbhr station is (Dij = nbhr_dist[j])

Then, each station has it's time update, it's GPS update(a.k.a incremental update), it's cross link update, and it's diffusion update as outlined in the time transfer paper. 

There is also a function second_cross_link which finds the T_g,B' which is shared in the second cross link measurements from the calculated values of psi, T_l,n, and D_ij




**Cross Link Covariance**

We also deal with the cross link covariance here (sigma_j in Eqn 38) . The paper doesn't specify what values they used but when we assumed it was small (0.001) our filter was VERY smug(small estimate covariance), but when we increased it the filter seemed to perform better. It is highlighted below in the cross-link-update function

In [146]:
class Station:
    def __init__(self,id):
        self.local_time = None
        self.global_est = 0 #\hat{T}_g,n
        self.id = id
        self.x = None
        self.P = None
        self.psi_n = None
        self.nbhrs = []
        self.nbhr_dist = {}

    def time_update(self, Q,delta_t = 60):
        # time update steps
        A = np.array([[1, delta_t], [0, 1]])
        self.x = A @ self.x
        self.P = A @ self.P @ A.T + Q

    def incremental_update(self,R,z):
    # Calculate pseudorange/range-rate residuals using Eqs. (23)-(25), add in speed of light
        N = int(len(z)/2)

        C = np.block([
                [np.ones((N, 1)), np.zeros((N, 1))],
                [np.zeros((N, 1)), np.ones((N, 1))]
            ])

        # Update the state and covariance estimate with Eqs. (26)-(28)
        K_n = self.P @ C.T @ np.linalg.inv((C @ self.P @ C.T) + R(N)) # Kalman gain matrix; R defined earlier
        x_hat_n = self.x + (K_n @ (z - C @ self.x))
        P_hat_n = (np.eye(2,2) - K_n @ C) @ self.P

        self.x = x_hat_n
        self.P = P_hat_n



    def crosslink_update(self,nbhr_time_ests,nbhr_cov_ests):
        H = np.array([[1, 0]])
        self.psi_n = np.copy(self.x)
        P_hat_n = np.copy(self.P)
        
        for nbhr_id in self.nbhrs:
            #Eqn 16
            z_jB = c*(self.local_time["B"]-nbhr_time_ests[nbhr_id] - self.nbhr_dist[nbhr_id])

            #Eqn 34
            z_jn = np.array([[z_jB + self.x[1][0]*(self.local_time["n"] - self.local_time["B"])]])
            

            #HERE IS THE CROSS LINK COVARIANCE!!
            sigma_j = 0.01
            R_j = np.array([[sigma_j + nbhr_cov_ests[nbhr_id][0][0]]])

            K_ij_n = P_hat_n @ H.T @ np.linalg.inv((H @ P_hat_n @ H.T) + R_j)
            self.psi_n = self.psi_n + (K_ij_n @ (z_jn - H @ self.psi_n))
            P_hat_n = (np.eye(2,2) - K_ij_n @ H) @ P_hat_n
        
        self.P = P_hat_n  
          
    #This simulated T_g,B' to send to another station using our T_g,n
    def second_cross_link(self,D_ij):
        return self.local_time["n"] - ((1/c)*self.psi_n[0][0]) + D_ij

    def diffusion_update(self,B,nbhr_time_est):
        #The nbhr_time_est are the T_g,B' from our neighbors
        my_T_hat_g_n = self.local_time["n"] - (1/c)*self.psi_n[0][0]

        #This is our extrapolation of T_g,n from T_g,B"
        T_hat_g_n = {nbhr_id:nbhr_time_est[nbhr_id]+(1-(self.x[1][0]/c))*(self.local_time['n'] - self.local_time['Bp']) for nbhr_id in self.nbhrs}
        
        #Actual DIffusion step
        self.global_est = sum([B[self.id][nbhr_id]*T_hat_g_n[nbhr_id] for nbhr_id in self.nbhrs]+[B[self.id][self.id]*my_T_hat_g_n])
        self.x = np.array([[c*(self.local_time["n"] - self.global_est)],self.x[1]])

**Measurement Simulation**

As opposed to simulating astrodynamics to generate measurements, we instead keep track of the true global time as well as the true bias and drift of each station. With this, we are able to simulate the psuedorange and psuedorange rate residual measurements by simply adding measurement noise, drawn from a zero mean distribution with our measurement covariance (R).

This computation to find the measurements is done below

In [147]:
def measurement_noise(R,N_i):
     return np.linalg.cholesky(R(N_i))@np.random.normal(np.array([[0 for _ in range(2*N_i)]]).T)

def GPS_measurements(true_bias, true_drift,R,N_i = 2):

     true_pr = np.array([[c*(true_bias[0])] for _ in range(N_i)])

     true_prr = np.array([[c*(true_drift[0])] for _ in range(N_i)])

     true_measurements = np.vstack((true_pr,true_prr))

     return true_measurements+measurement_noise(R,N_i)

**Single Filter Iteration**

This is where we simulate a single iteration of our filter. We have a function to setup the filter before the first iteration. This includes giving each station it's initial estimate as well as setting it's neighbors and neighbor distances. 

We also have the aforementioned functions to set the local times at each station. We assume that the algorithms take zero time and each cross link communication takes 3 seconds. We set the local times to match this. 

To run a single iteration of the filter we can set how many satellites are visible to each station (it defaults to 2 per station) and then we set the local times based off the true times and true biases and run each step. We return the measurements so we can plot them later

In [148]:
def filter_initialize(stations,adj_mat,x_initial,P_initial):
  M = len(stations)
  for i in range(M):
    stations[i].x = x_initial[i]
    stations[i].P = P_initial[i]

    for j in range(i+1,M):
        if adj_mat[i][j]!=0:
          stations[i].nbhrs.append(j)
          stations[j].nbhrs.append(i)
          stations[i].nbhr_dist[j] = adj_mat[i][j]
          stations[j].nbhr_dist[i] = adj_mat[i][j]

def new_local_times(true_time,true_bias,true_drift):
   T_A = true_time+true_bias
   T_B = T_A+3+(true_drift*3)
   T_n = T_B
   T_Bp = T_n+3+(true_drift*3)

   return {'A':T_A,'B':T_B,'n':T_n,'Bp':T_Bp}


def diffusion_filter_iteration(stations,Q,R,diff_weights, gps_measurements, cross_link_times, cross_link_cov, true_bias, true_drift,true_time,dt,N=None):
    if N is None: N = [2 for _ in range(len(stations))]

    #share first_cross_link
    measurements = []

    for station in stations:
      station.local_time = new_local_times(true_time,true_bias[station.id][0],true_drift[station.id][0])

      # time update step
      station.time_update(Q,delta_t = dt)

      z = gps_measurements[station.id]
      measurements.append(z)
      station.incremental_update(R,z)
    
      if cross_link_times:
          # initialize psi_n with x from step 2 (can choose from step 1 or 2)
          # station.psi_n = np.copy(station.x)
          time_ests = {nbhr_id:cross_link_times[nbhr_id] for nbhr_id in station.nbhrs}
          cov_ests = {nbhr_id:cross_link_cov[nbhr_id] for nbhr_id in station.nbhrs}
          station.crosslink_update(time_ests,cov_ests)
      else:
          station.psi_n = np.copy(station.x)

    for station in stations:
      time_ests_Bp = {nbhr_id:stations[nbhr_id].second_cross_link(station.nbhr_dist[nbhr_id]) for nbhr_id in station.nbhrs}
      station.diffusion_update(diff_weights, time_ests_Bp)

      #Update from T_g,n to T_g,A by adding time between
      station.global_est += 3

    time_ests_n = [station.global_est for station in stations]
    cov_ests_n = [station.P for station in stations]

    return measurements,time_ests_n, cov_ests_n